In [1]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')

In [2]:
#!pip install "dask[distributed]" --upgrade

In [3]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [4]:
df_DRUG_EXPOSURE = pd.read_sql('select * from `DRUG_EXPOSURE`', con=conn)

/tmp/ipykernel_19577/254203310.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_DRUG_EXPOSURE = pd.read_sql('select * from `DRUG_EXPOSURE`', con=conn)


In [5]:
print('Length - DRUG_EXPOSURE: ', len(df_DRUG_EXPOSURE))

Length - DRUG_EXPOSURE:  0


In [6]:
print(df_DRUG_EXPOSURE.head())

Empty DataFrame
Columns: [drug_exposure_id, person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_start_datetime, drug_exposure_end_date, drug_exposure_end_datetime, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, visit_detail_id, drug_source_value, drug_source_concept_id, route_source_value, dose_unit_source_value]
Index: []

[0 rows x 23 columns]


In [7]:
import MySQLdb
import pandas as pd

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

mimicivconn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='mimiciv')


In [8]:
df_prescriptions_mimiciv = pd.read_sql('select * from prescriptions', con=mimicivconn)

/tmp/ipykernel_19577/2993937686.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_prescriptions_mimiciv = pd.read_sql('select * from prescriptions', con=mimicivconn)


In [9]:
import dask.dataframe as dd
ddf_prescriptions_mimiciv = dd.from_pandas(df_prescriptions_mimiciv).compute()

In [10]:
df_prescriptions_mimiciv = ''

In [11]:
print(ddf_prescriptions_mimiciv.head())

   subject_id   hadm_id  pharmacy_id       poe_id  poe_seq order_provider_id  \
0    10000032  22595853     11700683  10000032-34     34.0            P76JEQ   
1    10000032  22595853     14779570  10000032-22     22.0            P76JEQ   
2    10000032  22595853     19796602  10000032-50     50.0            P260SK   
3    10000032  22595853     20256254  10000032-32     32.0            P76JEQ   
4    10000032  22595853     28781051  10000032-27     27.0            P76JEQ   

            starttime            stoptime drug_type  \
0 2180-05-07 01:00:00 2180-05-07 22:00:00      MAIN   
1 2180-05-07 00:00:00 2180-05-07 22:00:00      MAIN   
2 2180-05-08 08:00:00 2180-05-07 22:00:00      MAIN   
3 2180-05-07 01:00:00 2180-05-07 22:00:00      MAIN   
4 2180-05-07 00:00:00 2180-05-07 22:00:00      MAIN   

                          drug  ...     gsn          ndc  \
0                Acetaminophen  ...  004490  00904198861   
1  Sodium Chloride 0.9%  Flush  ...    <NA>            0   
2       

In [12]:
ddf_prescriptions_mimiciv['year'] = ddf_prescriptions_mimiciv['starttime'].dt.year
ddf_prescriptions_mimiciv['month'] = ddf_prescriptions_mimiciv['starttime'].dt.month
ddf_prescriptions_mimiciv['day'] = ddf_prescriptions_mimiciv['starttime'].dt.day

ddf_prescriptions_mimiciv['drug_exposure_start_date'] = pd.to_datetime(
    ddf_prescriptions_mimiciv[['year', 'month', 'day']],
    errors='coerce'
)

del ddf_prescriptions_mimiciv['year']
del ddf_prescriptions_mimiciv['month']
del ddf_prescriptions_mimiciv['day']

print(ddf_prescriptions_mimiciv[['drug_exposure_start_date']])


         drug_exposure_start_date
0                      2180-05-07
1                      2180-05-07
2                      2180-05-08
3                      2180-05-07
4                      2180-05-07
...                           ...
15416703               2145-11-03
15416704               2145-11-04
15416705               2145-11-10
15416706               2145-11-09
15416707               2145-11-09

[15416708 rows x 1 columns]


In [13]:
ddf_prescriptions_mimiciv['year'] = ddf_prescriptions_mimiciv['stoptime'].dt.year
ddf_prescriptions_mimiciv['month'] = ddf_prescriptions_mimiciv['stoptime'].dt.month
ddf_prescriptions_mimiciv['day'] = ddf_prescriptions_mimiciv['stoptime'].dt.day

ddf_prescriptions_mimiciv['drug_exposure_end_date'] = pd.to_datetime(
    ddf_prescriptions_mimiciv[['year', 'month', 'day']],
    errors='coerce'
)

del ddf_prescriptions_mimiciv['year']
del ddf_prescriptions_mimiciv['month']
del ddf_prescriptions_mimiciv['day']

print(ddf_prescriptions_mimiciv[['drug_exposure_end_date']])

         drug_exposure_end_date
0                    2180-05-07
1                    2180-05-07
2                    2180-05-07
3                    2180-05-07
4                    2180-05-07
...                         ...
15416703             2145-11-03
15416704             2145-11-11
15416705             2145-11-11
15416706             2145-11-09
15416707             2145-11-09

[15416708 rows x 1 columns]


In [14]:
ddf_prescriptions_mimiciv['verbatim_end_date'] = pd.NaT
print(ddf_prescriptions_mimiciv[['verbatim_end_date']])

         verbatim_end_date
0                      NaT
1                      NaT
2                      NaT
3                      NaT
4                      NaT
...                    ...
15416703               NaT
15416704               NaT
15416705               NaT
15416706               NaT
15416707               NaT

[15416708 rows x 1 columns]


In [15]:
ddf_prescriptions_mimiciv['drug_type_concept_id'] = 32838
print(ddf_prescriptions_mimiciv['drug_type_concept_id'])


0           32838
1           32838
2           32838
3           32838
4           32838
            ...  
15416703    32838
15416704    32838
15416705    32838
15416706    32838
15416707    32838
Name: drug_type_concept_id, Length: 15416708, dtype: int64


In [16]:
ddf_prescriptions_mimiciv['visit_detail_id'] = pd.NaT
print(ddf_prescriptions_mimiciv[['visit_detail_id']])

         visit_detail_id
0                    NaT
1                    NaT
2                    NaT
3                    NaT
4                    NaT
...                  ...
15416703             NaT
15416704             NaT
15416705             NaT
15416706             NaT
15416707             NaT

[15416708 rows x 1 columns]


In [17]:
provider_id = 1
ddf_prescriptions_mimiciv['order_provider_id'] = provider_id
print(ddf_prescriptions_mimiciv['order_provider_id'])

0           1
1           1
2           1
3           1
4           1
           ..
15416703    1
15416704    1
15416705    1
15416706    1
15416707    1
Name: order_provider_id, Length: 15416708, dtype: int64


In [18]:
ddf_prescriptions_mimiciv['stop_reason'] = pd.NaT
print(ddf_prescriptions_mimiciv[['stop_reason']])

         stop_reason
0                NaT
1                NaT
2                NaT
3                NaT
4                NaT
...              ...
15416703         NaT
15416704         NaT
15416705         NaT
15416706         NaT
15416707         NaT

[15416708 rows x 1 columns]


In [19]:
ddf_prescriptions_mimiciv['refills'] = pd.NaT
print(ddf_prescriptions_mimiciv[['refills']])

         refills
0            NaT
1            NaT
2            NaT
3            NaT
4            NaT
...          ...
15416703     NaT
15416704     NaT
15416705     NaT
15416706     NaT
15416707     NaT

[15416708 rows x 1 columns]


In [20]:
ddf_prescriptions_mimiciv['days_supply'] = pd.NaT
print(ddf_prescriptions_mimiciv[['days_supply']])

         days_supply
0                NaT
1                NaT
2                NaT
3                NaT
4                NaT
...              ...
15416703         NaT
15416704         NaT
15416705         NaT
15416706         NaT
15416707         NaT

[15416708 rows x 1 columns]


In [21]:
ddf_prescriptions_mimiciv['sig'] = pd.NaT
print(ddf_prescriptions_mimiciv[['sig']])

         sig
0        NaT
1        NaT
2        NaT
3        NaT
4        NaT
...       ..
15416703 NaT
15416704 NaT
15416705 NaT
15416706 NaT
15416707 NaT

[15416708 rows x 1 columns]


In [22]:
ddf_prescriptions_mimiciv['lot_number'] = pd.NaT
print(ddf_prescriptions_mimiciv[['lot_number']])

         lot_number
0               NaT
1               NaT
2               NaT
3               NaT
4               NaT
...             ...
15416703        NaT
15416704        NaT
15416705        NaT
15416706        NaT
15416707        NaT

[15416708 rows x 1 columns]
